In [1]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)

from compute_ET_green import calculate_band_std_dev

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple, Union, Optional

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

### Postprocess and export ET Blue

In [5]:
TIME_STEPS = 36
TIME_STEP_TYPE = "dekadal"
YEARS = range(2022, 2023)

In [6]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [7]:
ET_BLUE_RAW_COLLECTION = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_wapor_10m_zh_dekadal_2022").map(lambda img: back_to_float(img, 100))

In [8]:
ET_GREEN_COLLECTION = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_green_wapor_jurisdiction_zh_dekadal_2022").map(lambda img: back_to_float(img, 100))

In [9]:
def process_and_export_et_blue_postprocessed(
    et_blue_raw_list: ee.List,
    et_green_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
) -> List[ee.batch.Task]:
    """
    Process and export post-processed ET blue images for a given year.

    Args:
        et_blue_raw_list (ee.List): List of raw ET blue images
        et_green_list (ee.List): List of ET green images
        year (int): Year to process
        year_index (int): Index of the year (e.g., 2022 would be 4 if starting from 2018)
        aoi (ee.Geometry): Area of interest to process
        time_steps (int): Number of time steps (default 36 for dekadal)
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        List[ee.batch.Task]: List of export tasks

    Raises:
        ValueError: If time_steps is not 12 or 36, or if time_step_type is invalid
    """
    if time_steps not in [12, 36]:
        raise ValueError("time_steps must be either 12 or 36")

    if time_step_type not in ["monthly", "dekadal"]:
        raise ValueError("time_step_type must be either 'monthly' or 'dekadal'")

    tasks = []
    et_blue_previous = None

    for i in range(time_steps):
        # Get current images
        et_green = ee.Image(et_green_list.get(i))
        et_blue_present = ee.Image(et_blue_raw_list.get(i))

        # Initialize previous for first iteration
        if et_blue_previous is None:
            et_blue_previous = et_blue_present

        # Process ET blue
        et_blue = _process_et_blue_image(
            et_blue_present=et_blue_present,
            et_blue_previous=et_blue_previous,
            et_green=et_green,
        )

        # Store current processed image for next iteration
        et_blue_previous = et_blue.select("ET_blue")

        # Generate time step name based on type
        time_step_name = _get_time_step_name(i, time_step_type)

        # Generate export task
        task = _create_export_task(
            et_blue=et_blue,
            year=year,
            time_step_name=time_step_name,
            time_step_type=time_step_type,
            aoi=aoi,
        )

        tasks.append(task)

    return tasks


def _process_et_blue_image(
    et_blue_present: ee.Image,
    et_blue_previous: ee.Image,
    et_green: ee.Image,
) -> ee.Image:
    """
    Process a single ET blue image with temporal post-processing.

    Args:
        et_blue_present (ee.Image): Current ET blue image
        et_blue_previous (ee.Image): Previous ET blue image
        et_green (ee.Image): Current ET green image

    Returns:
        ee.Image: Processed ET blue image with volumetric band
    """
    # Calculate threshold from ET green
    threshold = calculate_band_std_dev(et_green, "ET_green")

    # Post process using the previous processed image
    et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

    # Compute and add volumetric band
    et_blue_m3 = compute_volumetric_et_blue(et_blue)
    et_blue = et_blue.addBands(et_blue_m3)

    # Convert to int for storage
    return back_to_int(et_blue, 100)


def _get_time_step_name(index: int, time_step_type: str) -> str:
    """
    Generate the time step name based on index and type.

    Args:
        index (int): Current time step index
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        str: Formatted time step name
    """
    if time_step_type == "dekadal":
        month = index // 3 + 1
        dekad = index % 3 + 1
        return f"{month:02d}_D{dekad}"
    else:  # monthly
        month = index + 1
        return f"{month:02d}"


def _create_export_task(
    et_blue: ee.Image,
    year: int,
    time_step_name: str,
    time_step_type: str,
    aoi: ee.Geometry,
) -> ee.batch.Task:
    """
    Create an export task for a post-processed ET blue image.

    Args:
        et_blue (ee.Image): ET blue image to export
        year (int): Processing year
        time_step_name (str): Formatted time step name
        time_step_type (str): Type of time step
        aoi (ee.Geometry): Area of interest

    Returns:
        ee.batch.Task: Export task
    """
    task_name = (
        f"ET_blue_postprocessed_refactored_{time_step_type}_{year}-{time_step_name}"
    )

    asset_id = (
        f"projects/thurgau-irrigation/assets/Thurgau/"
        f"ET_blue_postprocessed_wapor_{time_step_type}_refactored_2018-2023/{task_name}"
    )

    return export_image_to_asset(
        image=et_blue, asset_id=asset_id, task_name=task_name, year=year, aoi=aoi
    )

In [10]:
# all_tasks = []

# for year in YEARS:

#     et_blue_raw_collection_list = ET_BLUE_RAW_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     et_green_collection_list = ET_GREEN_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     tasks = process_and_export_et_blue_postprocessed(
#         et_blue_raw_list=et_blue_raw_collection_list,
#         et_green_list=et_green_collection_list,
#         year=year,
#         aoi=AOI,
#         time_steps=TIME_STEPS,
#         time_step_type=TIME_STEP_TYPE,
#     )

#     print(f"Year {year} processing complete. Started {len(tasks)} tasks.")

#     all_tasks.extend(tasks)

# print(f"Total tasks: {len(all_tasks)}")

### Sanity check

In [17]:
et_blue_postprocessed = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_wapor_monthly_2018-2023").map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(et_blue_postprocessed)
ee_utils.print_value_ranges(et_blue_postprocessed)

Dates of images in the collection:
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
2021-01-01
2021-02-01
2021-03-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
2023-01-01
2023-02-01
2023-03-01
2023-04-01
2023-05-01
2023-06-01
2023-07-01
2023-08-01
2023-09-01
2023-10-01
2023-11-01
2023-12-01
Image 1: Min = 0.00, Max = 45.75
Image 2: Min = 0.00, Max = 19.91
Image 3: Min = 0.00, Max = 34.89
Image 4: Min = 0.00, Max = 50.68
Image 5: Min = 0.00, Max = 57.31
Image 6:

In [12]:
et_blue_postprocessed.first().bandNames().getInfo()

['ET_blue', 'ET_blue_m3']

In [22]:
Map = geemap.Map()

et_blue_postprocessed_list = et_blue_postprocessed.toList(72)

image = ee.Image(et_blue_postprocessed_list.get(19))

vis_params = {
    "bands" : ["ET_blue"], 
    "min" : 0,
    "max" : 30,
    "palette" : ["blue", "lightblue", "green", "yellow", "orange", "red"]
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET blue raw")

Map

Map(center=[47.41511720684401, 8.655992127451274], controls=(WidgetControl(options=['position', 'transparent_b…